# Pyomo Homework 1

In [43]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()
    # helper.download_data(['knapsack_data.xlsx','knapsack_data.csv'])
else:
    # run solutions locally (TA/instructor testing mainly)
    import idaes

--2023-01-22 16:51:06--  https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6463 (6.3K) [text/plain]
Saving to: ‘helper.py.3’

helper.py.3         100%[===================>]   6.31K  --.-KB/s    in 0s      

2023-01-22 16:51:06 (42.9 MB/s) - ‘helper.py.3’ saved [6463/6463]

IDAES found! No need to install.
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfully installed
Installing glpk via apt-get...


In [44]:
## IMPORT LIBRARIES
import pyomo.environ as pyo
import pandas as pd

Special thanks to the Pyomo team for create these excercises as part of their excellent PyomoFest workshop.

## 1 Pyomo Fundamentals

### 1.1 Knapsack example

You want to fill a knapsack (a.k.a. bag). You can choose from a hammer, wrench, screwdriver, and towel. Each item has a different weight and value. You want to maximize the value (benefit) of the collection of items constrained by a total weight limit. Let's formulate this as an optimization problem.

**Sets**

$$\mathcal{A} = \{\text{hammer},~\text{wrench},~\text{screwdriver},~\text{towel} \}$$  

**Parameters (Data)**

Let $b_i$ and $w_i$ represent the benefit and weight of item $i$, respectfully.

| Item ($i$)  | Benefit ($b_i$) | Weigth ($w_i$) |
| ----------- | ----------- | ----------- |
| hammer      | 8      | 5|
| wrench   | 3        | 7 |
| screwdriver  | 6 | 4        |
| towel   | 11  | 3 |

Let $W_{max} = 14$ be the maximum weight.

**Variables**

Let $x_i \in \{0,1\}$ (binary) represent whether or not we include item $i$ in the knapsack. For now, we will consider only being able to choose either none or one of each item.

**Objective and Constraints**

$$
\begin{equation} 
\begin{split}
\max_{x} \quad & \sum_{i\in{\mathcal{A}}}b_i x_i \\
\text{s.t.} \quad & \sum_{i\in{\mathcal{A}}}w_ix_i \leq W_{max} \\
& x_i \in \{0,1\}, \quad \forall i \in \mathcal{A}
\end{split}
\end{equation}
$$


**Pyomo**

Solve the knapsack problem given below using GLPK and answer the following questions:

1. Which items are acquired in the optimal solution?

2. Why does this solution make sense? (Write ~2 sentences.)

In [45]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

model.obj = pyo.Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = pyo.maximize )

model.weight_con = pyo.Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# Add your solution here
"""
!!! if you are on the local system (anaconda), you need to install glpk
manually using the following command:
Anaconda Prompt: conda install -c conda-forge glpk
"""
# Specify the solver
opt = pyo.SolverFactory('glpk')
# Solve
opt_success = opt.solve(model)
#
model.pprint()
#
model.display()


1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf 

**Question Answers**

1. *As $x_i$ is a binary value, the items with value equal to one are included in optimal solution.Here, they are Hammer, Screwdriver and towel.*

2. *As these three items (Hammer, Screwdriver and towel) have the high benefit with low weight while the wrench has the least benefit with high weight.*

### 1.2 Knapsack example with improve printing

Complete the missing lines in the code below to produce formatted output: print the total weight, the value of the items selected (the objective), and the items acquired in the optimal solution. Note, the Pyomo value function should be used to get the floating point value of Pyomo modeling components (e.g., `print(value(model.x[i])`).

In [46]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

model.obj = pyo.Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = pyo.maximize )

model.weight_con = pyo.Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = pyo.SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
# Add your solution here
print("The total weight of items aquired in the optimal solution: ", round(total_weight, 2))
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    # Add your solution here
    print('%12s %12s' % (model.x[i], pyo.value(model.x[i])))
print('-------------------------')

The total weight of items aquired in the optimal solution:  12.0
        Item     Selected
   x[hammer]          1.0
   x[wrench]          0.0
x[screwdriver]          1.0
    x[towel]          1.0
-------------------------


### 1.3 Changing data

Using your code from **Question 1.2**, if we were to increase the value of the wrench, at what point would it become selected as part of the optimal solution?

In [47]:
# Add your solution here
"""
Here, a loop is provided to iteratively increase the 
value of the wrench untill selected items value of wrench
become equal to 1.
"""
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

while True:
  model.obj = pyo.Objective(
      expr = sum( b[i]*model.x[i] for i in A ), 
      sense = pyo.maximize )

  model.weight_con = pyo.Constraint(
      expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

  opt = pyo.SolverFactory('glpk')
  opt_success = opt.solve(model)

  total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
  if pyo.value(model.x['wrench']) == 1:
    break
  b['wrench'] += 1


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [48]:
total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
# Add your solution here
print("The total weight of items aquired in the optimal solution: ", round(total_weight, 2))
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    # Add your solution here
    print('%12s %12s' % (model.x[i], pyo.value(model.x[i])))
print('-------------------------')

The total weight of items aquired in the optimal solution:  14.0
        Item     Selected
   x[hammer]          0.0
   x[wrench]          1.0
x[screwdriver]          1.0
    x[towel]          1.0
-------------------------


**Question Answer**

*By increasing the value, the items wrench, screwdriver and towl become optimal solution with total weight of 14 which meet the requirement of the constraint.*

### 1.4 Loading data from Excel

In the code above, the data is hardcoded at the top of the file. Instead of hardcoding the data, use Python to load the data from a difference source. You may use Pandas to load data from 'knapsack_data.xlsx' into a dataframe. You will then need to write code to obtain a dictionary from the dataframe.

In [49]:
df_items = pd.read_excel('https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/data/knapsack_data.xlsx', sheet_name='data', header=0, index_col=0)
W_max = 14

A = df_items.index.tolist()
# Add your solution here
### Solution
"""
Create a dictionary from the data using pandas, the output should
be like the following:
{
'Benefit': {'hammer': 8, 'wrench': 3, 'screwdriver': 6, 'towel': 11}, 
'Weight': {'hammer': 5, 'wrench': 7, 'screwdriver': 4, 'towel': 3}
}
"""
dict_items = df_items.to_dict()
print(dict_items)
###
model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

model.obj = pyo.Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = pyo.maximize )

model.weight_con = pyo.Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = pyo.SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
print('Total Weight:', total_weight)
print('Total Benefit:', pyo.value(model.obj))

print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if pyo.value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

{'Benefit': {'hammer': 8, 'wrench': 3, 'screwdriver': 6, 'towel': 11}, 'Weight': {'hammer': 5, 'wrench': 7, 'screwdriver': 4, 'towel': 3}}
Total Weight: 14.0
Total Benefit: 25.0
        Item     Selected
      hammer           No
      wrench          Yes
 screwdriver          Yes
       towel          Yes
-------------------------


### 1.5 NLP vs. MIP

Solve the knapsack problem with IPOPT instead of glpk. Print the solution values for model.x. What happened? Why?

*Hint*: Switch `glpk` to `ipopt` in the call to `SolverFactory`.

In [50]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

model.obj = pyo.Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = pyo.maximize )

model.weight_con = pyo.Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# Add your solution here
###
opt = pyo.SolverFactory('ipopt')
opt_success = opt.solve(model)
###
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value              : Upper : Fixed : Stale : Domain
             hammer :     0 :                1.0 :     1 : False : False : Binary
        screwdriver :     0 :                1.0 :     1 : False : False : Binary
              towel :     0 :                1.0 :     1 : False : False : Binary
             wrench :     0 : 0.2857142884855869 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                           

**Question Answers**

Fisrt, let's see what are the differences between glpk and ipopt:

ipopt: This is a solver for the nonlinear problems

GLPK: This is a solver for linear problems which detects a result as binary format. 

For this reason, in the first part we see the solution as {0, 1} but in the second part the values are anything between [0, 1].


## More Pyomo Fundamentals

### 2.1 Knapsack problem with rules

Rules are important for defining indexed constraints, however, they can also be used for single (i.e. scalar) constraints. Reimplement the knapsack model from **Question 1.1** using rules for the objective and the constraints.

In [56]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

# Add your solution here
###
"""
In the previous part the objective and constraints are defined using 'expr',
however, this is useful if we have small indexed formulation, generally is better
to use rules (pre-defined functions), to implement.
"""
def objective_rule(model):
  return sum(b[i]*model.x[i] for i in A)

def constraint_rule(model):
  return sum( w[i]*model.x[i] for i in A ) <= W_max 

model.obj = pyo.Objective(rule = objective_rule, sense = pyo.maximize )

model.weight_con = pyo.Constraint(rule = constraint_rule )
###
# Specify the solver
opt = pyo.SolverFactory('glpk')
# Solve
opt_success = opt.solve(model)
#
model.pprint()
"""
As it can be seen by changing the rules the results are the same!
"""

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf 

### 2.2 Integer formulation of the knapsack problem

Consider again the knapsack problem. Assume now that we can acquire multiple items of the same type. In this new formulation, $x_i$ is now an integer variable instead of a binary variable. One way to formulate this problem is as follows:

$$
\begin{equation} 
\begin{split}
\max_{x} \quad & \sum_{i\in{\mathcal{A}}}b_i x_i \\
\text{s.t.} \quad & \sum_{i\in{\mathcal{A}}}w_i x_i \leq W_{max} \\
 & x_i=\sum_{j=0}^Njq_{i,j}, \quad \forall i \in \mathcal{A} \\
 & 0 \leq x_i \leq N, \quad \forall i \in \mathcal{A} \\
 & q_{i,j} \in \{0,1\}, \quad \forall i \in \mathcal{A}, j \in \{0,...,N\}
\end{split}
\end{equation}
$$

One could optionally add the following constraint select only one $q_{i,j}$ for each $i$, although it is not stricly neccessary to yield an integer solution.
$$
\begin{equation}
\sum_{j=0}^N q_{i,j} = 1, \quad \forall i \in \mathcal{A}
\end{equation}
$$

Starting with your code from **Question 2.1**, implement this new formulation and solve. Is the solution surprising?

In [80]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14
N = range(6) # create a list from 0-5

model = pyo.ConcreteModel()
# !!! I think here we should add the boundaries for model.x
model.x = pyo.Var(A , bounds=(0, 6))
model.q = pyo.Var(A, N, within=pyo.Binary )

def obj_rule(m):
    return sum( b[i]*m.x[i] for i in A )
model.obj = pyo.Objective(rule=obj_rule, sense = pyo.maximize )

def weight_con_rule(m):
    return sum( w[i]*m.x[i] for i in A ) <= W_max
model.weight_con = pyo.Constraint(rule=weight_con_rule)

# Add your solution here
def second_con_rule(m, a):
    return sum( m.q[a,n] for n in N) == 1
model.second_con = pyo.Constraint(A, rule=second_con_rule)

opt = pyo.SolverFactory('glpk')
opt_success = opt.solve(model)
#
model.pprint()
#
total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
# Add your solution here
print("The total weight of items aquired in the optimal solution: ", round(total_weight, 2))
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    # Add your solution here
    print('%12s %12s' % (model.x[i], pyo.value(model.x[i])))
print('-------------------------')

5 Set Declarations
    q_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : q_index_0*q_index_1 :   24 : {('hammer', 0), ('hammer', 1), ('hammer', 2), ('hammer', 3), ('hammer', 4), ('hammer', 5), ('wrench', 0), ('wrench', 1), ('wrench', 2), ('wrench', 3), ('wrench', 4), ('wrench', 5), ('screwdriver', 0), ('screwdriver', 1), ('screwdriver', 2), ('screwdriver', 3), ('screwdriver', 4), ('screwdriver', 5), ('towel', 0), ('towel', 1), ('towel', 2), ('towel', 3), ('towel', 4), ('towel', 5)}
    q_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}
    q_index_1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    second_con_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : 

In [79]:
total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
print(total_weight)

14.000000000000009


**Question Answer**

*I think the solution makes sense as the towel has the highest value with least weight, and as the repetition is allowed the algorithm should converge to this solution*